# P3 - Data Wrangling with MongoDB

### by Satvik Sachdev 

# Project Summary

Map Used: Phoenix, Arizona
Objective: 
    - Work with OpenStreetMap dataset
    - Assess the quality of the data for validity, accuracy, completeness, consistency and uniformity.
    - Parse and gather data from popular file formats such as .json, .xml, .csv
    - Process data from large files
    - Learn how to store, query, and aggregate data using MongoDB
    

# 1. Data Audit

### 1.1 Unique Tags

Task: How to parse the file and find all unique tags?

Challenges:
    - File size > 2GB
    - Cannot load entire file into memory.
    - Therefore, use iterative parser over tree based parser to load one element at a time.
    
Findings:
    - Differetnt types of tags
    - Count of tags

mapparser.py is used count the number of unique tags.

In [ ]:
OSMFILE = 'data/phoenix_osm_dataset'
OSMFILE

In [ ]:
import mapparser
mapparser.count_tags(OSMFILE)

### 1.2 Check for Patterns

Use process_map from tags.py to find any problematic data.

In [ ]:
import tags
tags.process_map(OSMFILE)

### 1.3 List all Users

Task:
    - Find the number of unique users who have contributed to the dataset

Use process_map from users.py to generate a list of all users who have contributed to the dataset.

In [ ]:
import users
all_users = users.process_map(OSMFILE)
all_users

# 2. Probelms Encountered

Overview: 
    
Task:
        - audit dataset to reflect the changes needed to fix the unexpected street types to the appropriate ones in the expected list.
        - Create a function to fix the street names.

Challenges:
    - Multiple errors. 
    - Street names are differently abbreviated leading to inconsistency.
    - Each user types in things differently.


### 2.1 Street Names

In [5]:
from audit_file import *

Let's have a look at some of the erroreous data.

In [6]:
street_data = audit(OSMFILE)

from pprint import pprint
pprint(dict(street_data))

{'100': set(['East Elwood Street Suite 100']),
 '101': set(['Grand Avenue #101', 'N 27th Ave Ste 101']),
 '102': set(['North 93rd Avenue #102', 'W Lake Pleasant Pkwy # 102']),
 '103': set(['E Greenway Pkwy #103']),
 '105': set(['N Scottsdale Rd #105', 'North 90th Street #105']),
 '108': set(['W Elliot Rd #108']),
 '1081': set(['N. 55th Ave #1081']),
 '110': set(['E 5th Ave #110', 'East Doubletree Ranch Road #110']),
 '117': set(['N 51st Ave #117']),
 '120': set(['North 16th Avenue #120']),
 '133': set(['West Indian School Road #133']),
 '1400-1532': set(['N. Central Avenue, Suite 1400-1532']),
 '148': set(['S 48th St #148']),
 '15935': set(['E Thomas Road #15935']),
 '170-A': set(['E. Chandler Blvd. #170-A']),
 '200': set(['East Elwood Street Suite 200',
             'East Marilyn Road Building 4 Suite 200',
             'W Union Hills Dr., Building D, Suite 200']),
 '201': set(['West Ray Road, Suite 201']),
 '202': set(['N Dysart Rd #202']),
 '208': set(['N 19th Ave Suite 208']),
 '21

Now, I will try to update the names using the mapping created inside audit_file to fix the erroneous data.

In [7]:
for street_type, ways in street_data.iteritems():
    for name in ways:
        better_name = update_name(name, mapping)
        print name, "=>", better_name

East Pinnacle Peak Road #B119 => East Pinnacle Peak Road #B119
N. Hayden Road, Suite 210 => N. Hayden Road, Suite 210
218 3rd Ave#1515 => 218 3rd Ave#1515
West Indian School Road #133 => West Indian School Road #133
N 32nd St Lot 21 => N 32nd Street Lot 21
North 27th Avenue #401 => North 27th Avenue #401
E Thomas Road #15935 => E Thomas Road #15935
N. Central Avenue, Suite 1400-1532 => N. Central Avenue, Suite 1400-1532
East Rio Salado Pkwy => East Rio Salado Parkway
North 16th Avenue #120 => North 16th Avenue #120
W Broadway Rd => W Broadway Road
N Scottsdale Rd => N Scottsdale Road
N Hayden Rd => N Hayden Road
W Camelback Rd => W Camelback Road
Manor Rd => Manor Road
E Baseline Rd => E Baseline Road
North Coyote Lakes Parkwway => North Coyote Lakes Parkwway
Tatum Blvd. Suite 3031 => Tatum Blvd. Suite 3031
North 28th Drive, Suite A-110 => North 28th Drive, Suite A-110
North 25th Glen => North 25th Glen
N. 55th Ave #1081 => N. 55th Avenue #1081
North Paradise Village Parkway West => No

In the above output we can see that we are able to achieve a desrired consistancy to make this data meaningful.

### 2.2 Zip Codes

In [10]:
from audit_file import *

In [12]:
from pprint import pprint
# zipcode_info = audit_zip(OSMFILE)

pprint(dict(zipcode_info))

{'19': set(['19971']),
 '21': set(['21013']),
 '29': set(['29801']),
 '75': set(['75201']),
 '8': set(['8']),
 '82': set(['82158', '82381']),
 '83': set(['83003']),
 '85': set(['85003',
            '85003-1076',
            '85003-1135',
            '85003-1333',
            '85003-1376',
            '85004',
            '85004-1118',
            '85004-1155',
            '85004-1316',
            '85004-1334',
            '85004-1336',
            '85004-1418',
            '85004-1455',
            '85004-1506',
            '85004-1640',
            '85004-1722',
            '85004-1820',
            '85004-1873',
            '85004-4527',
            '85006',
            '85006-3651',
            '85007',
            '85007-1909',
            '85007-2101',
            '85007-2121',
            '85007-2126',
            '85007-2129',
            '85007-2309',
            '85007-2419',
            '85007-2604',
            '85007-2607',
            '85007-2616',
            '85007-3232

After doing some research, I found out that the possible zipcode range is between 85001-86556.
Looking at the output above, I see many wrong values which need to be fixed.

I have created an update_zip function which fixes the following:
    - ensures that zipcode ranges in the corrent range
    - ensures the valid len of the zipcode
    - ensures that zipcode contains only numbers 

In [46]:
def update_zip(zipcode):
    zipcode.strip()
    if "AZ" in zipcode:
        zipcode = zipcode.replace("AZ", "")
    if len(zipcode) < 5:
        zipcode = None
    if zipcode is not None and len(zipcode) == 5:
        if zipcode < 85001 or zipcode > '86556':
            zipcode = None
    return zipcode

In [49]:
for street_type, ways in zipcode_info.iteritems():
    for zipcode in ways:
        better_zipcode = update_zip(zipcode)
        print zipcode, "=>", better_zipcode

AZ 85033 =>  85033
AZ 85015 =>  85015
AZ 85014 =>  85014
AZ 85351 =>  85351
AZ 85050 =>  85050
AZ 85374 =>  85374
AZ 85256 =>  85256
AZ 85255 =>  85255
AZ 85254 =>  85254
AZ 85253 =>  85253
AZ 85018 =>  85018
AZ 85251 =>  85251
AZ 85373 =>  85373
AZ 85034 =>  85034
AZ 85310 =>  85310
AZ 85338 =>  85338
AZ 85339 =>  85339
AZ 85257 =>  85257
AZ 85250 =>  85250
AZ 85375 ‎ =>  85375 ‎
AZ 85007 =>  85007
AZ 85004 =>  85004
AZ 85301 =>  85301
AZ 85008 =>  85008
AZ => None
AZ 85023 =>  85023
AZ 85226 =>  85226
AZ 85045 =>  85045
AZ 85042 =>  85042
AZ 85375 =>  85375
AZ 85383 =>  85383
AZ 85016 =>  85016
AZ 85281 =>  85281
AZ 85282 =>  85282
AZ 85283 =>  85283
21013 => 21013
19971 => 19971
29801 => 29801
99501 => None
98409 => None
98101 => None
75201 => 75201
8 => None
82381 => 82381
82158 => 82158
83003 => 83003
93702 => None
92509 => None
92127 => None
97701 => None
85023-2577 => 85023-2577
85023-8208 => 85023-8208
85392 => 85392
85248 => 85248
85023-8205 => 85023-8205
85032-7702 => 85032-7

### 2.3 Prepare data for database insertion

The following measures need to be done in order to insert data:
    - Process only "node" and "way" tags
    - Convert attrbutes of "node" and "ways" into key->value pairs( exceptions: latitude, longitude, created )
    - convert "addr:" => "address"
    - ignore tags which just contain ":"
    - ignore tags which separate type of street.

After applying the abovementioned steps, I will use process_map function to convert thet file from XML into JSON.

In [154]:
# -*- coding: utf-8 -*-
import xml.etree.cElementTree as ET
import pprint
import re
import codecs
import json

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
address_regex = re.compile(r'^addr\:')
street_regex = re.compile(r'^street')

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]


def shape_element(element):
    node = {}
    if element.tag == "node" or element.tag == "way" :
        # YOUR CODE HERE
        node['type'] = element.tag
        # initialize empty address
        address = {}
        # parsing through attributes
        for a in element.attrib:
            if a in CREATED:
                if 'created' not in node:
                    node['created'] = {}
                node['created'][a] = element.get(a)
            elif a in ['lat', 'lon']:
                continue
            else:
                node[a] = element.get(a)
        # populate position
        if 'lat' in element.attrib and 'lon' in element.attrib:
            node['pos'] = [float(element.get('lat')), float(element.get('lon'))]

        # parse second-level tags for nodes
        for e in element:
            # parse second-level tags for ways and populate `node_refs`
            if e.tag == 'nd':
                if 'node_refs' not in node:
                    node['node_refs'] = []
                if 'ref' in e.attrib:
                    node['node_refs'].append(e.get('ref'))

            # throw out not-tag elements and elements without `k` or `v`
            if e.tag != 'tag' or 'k' not in e.attrib or 'v' not in e.attrib:
                continue
            key = e.get('k')
            val = e.get('v')

            # skip problematic characters
            if problemchars.search(key):
                continue

            # parse address k-v pairs
            elif address_regex.search(key):
                key = key.replace('addr:', '')
                address[key] = val

            # catch-all
            else:
                node[key] = val
        # compile address
        if len(address) > 0:
            node['address'] = {}
            street_full = None
            street_dict = {}
            street_format = ['prefix', 'name', 'type']
            # parse through address objects
            for key in address:
                val = address[key]
                if street_regex.search(key):
                    if key == 'street':
                        street_full = val
                    elif 'street:' in key:
                        street_dict[key.replace('street:', '')] = val
                else:
                    node['address'][key] = val
            # assign street_full or fallback to compile street dict
            if street_full:
                node['address']['street'] = street_full
            elif len(street_dict) > 0:
                node['address']['street'] = ' '.join([street_dict[key] for key in street_format])
        return node
    else:
        return None


def process_map(file_in, pretty = False):
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data

data = process_map(OSMFILE, False)

# 3. Data Overview with MongoDB

## Import Data

In [171]:
from pymongo import MongoClient

db_name = 'map'

# Connect to Mongo DB
client = MongoClient("mongodb://localhost:27017")
db = client[db_name]

In [173]:
phoenix = db['map']
phoenix


Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'map'), u'map')

File Size

In [167]:
import os

In [174]:
 # convert from bytes to megabytes
print 'Original OSM file is {} MB'.format(os.path.getsize(data)/1.0e6)

# convert from bytes to megabytes
print 'New JSON file is {} MB'.format(os.path.getsize(data + ".json")/1.0e6) 

Original OSM file is 396.415019 MB
New JSON file is 442.342844 MB


### 3.1 Number of Documents

In [175]:
print "Total Number of Documents: {}".format(db['map'].find().count())

Total Number of Documents: 1896364


### 3.2 Number of Nodes

In [176]:
print "Number of nodes:",phoenix.find({'type':'node'}).count()

Number of nodes: 1661886


### 3.3 Number of ways

In [178]:
print "Number of ways:", phoenix.find({'type':'way'}).count()

Number of ways: 234412


### 3.4 Number of Unique Users

In [188]:
len(phoenix.distinct("created.user"))

1022

### 3.5 Top 5 Contributors

In [197]:
from pprint import pprint
result = phoenix.aggregate( [
                            { "$group" : {"_id" : "$created.user", "count" : { "$sum" : 1} } },
                            { "$sort" : {"count" : -1} }, 
                            { "$limit" : 5 } ] )

pprint(list(result))

[{u'_id': u'Dr Kludge', u'count': 894474},
 {u'_id': u'TheDutchMan13', u'count': 231185},
 {u'_id': u'Adam Martin', u'count': 155409},
 {u'_id': u'namannik', u'count': 40850},
 {u'_id': u'kghazi', u'count': 37019}]


### 3.6 Number of Cafes in Phoenix

In [204]:
phoenix.find({'amenity': "cafe"}).count()

131

# 4. Further Data Exploration with MongoDB

### 4.1 Users with only 1 entry

In [208]:
users = phoenix.aggregate( [
        { "$group" : {"_id" : "$created.user", "count" : { "$sum" : 1} } },
        { "$group" : {"_id" : "$count", "num_users": { "$sum" : 1} } },
        { "$sort" : {"_id" : 1} },
        { "$limit" : 1} ] )
pprint(list(users))

[{u'_id': 1, u'num_users': 234}]


### 4.2 Top 10 cuisines enjoyed in Phoenix

In [211]:
cuisine = phoenix.aggregate([
                        {"$match":{"amenity":{"$exists":1}, "amenity":"restaurant",}},      
                        {"$group":{"_id":{"Food":"$cuisine"}, "count":{"$sum":1}}},
                        {"$project":{"_id":0, "Food":"$_id.Food", "Count":"$count"}},
                        {"$sort":{"Count":-1}}, 
                        {"$limit":11}
                    ])
pprint(list(cuisine))

[{u'Count': 318, u'Food': None},
 {u'Count': 59, u'Food': u'mexican'},
 {u'Count': 38, u'Food': u'pizza'},
 {u'Count': 33, u'Food': u'american'},
 {u'Count': 22, u'Food': u'italian'},
 {u'Count': 20, u'Food': u'sandwich'},
 {u'Count': 20, u'Food': u'burger'},
 {u'Count': 17, u'Food': u'chinese'},
 {u'Count': 9, u'Food': u'regional'},
 {u'Count': 7, u'Food': u'sushi'},
 {u'Count': 7, u'Food': u'steak_house'}]


### 4.3 Top buildings

In [214]:
buildings = phoenix.aggregate([
                        {"$match": {'building': {"$exists": 1}}}, 
                        {"$group": {'_id': '$building', 'count': {"$sum": 1}}}, 
                        {"$sort": {'count': -1}}, 
                        {"$limit": 6}
                    ])
pprint(list(buildings))

[{u'_id': u'yes', u'count': 25047},
 {u'_id': u'house', u'count': 7399},
 {u'_id': u'roof', u'count': 3691},
 {u'_id': u'apartments', u'count': 2209},
 {u'_id': u'commercial', u'count': 1932},
 {u'_id': u'residential', u'count': 1573}]


### 4.4 Top Amenities

In [216]:
amenities = phoenix.aggregate([
                        {"$match": {'amenity': {"$exists": 1}}}, 
                        {"$group": {'_id': '$amenity', 'count': {"$sum": 1}}}, 
                        {"$sort": {'count': -1}}, 
                        {"$limit": 6}
                    ])
pprint(list(amenities))

[{u'_id': u'parking', u'count': 3204},
 {u'_id': u'fast_food', u'count': 779},
 {u'_id': u'school', u'count': 772},
 {u'_id': u'place_of_worship', u'count': 747},
 {u'_id': u'restaurant', u'count': 646},
 {u'_id': u'fuel', u'count': 570}]


# 5.Conclusion

Improvement Ideas:
    - During data auditing phase, it was clear that data comprises of several errors. A common platform 
    can be provided to make the data entry process smoother. With a platform, minor errors such as 
    having alphabets in numeric zipcodes can be programmatically avoided. By fillig up a form on a platform
    the user can be given guidance on what data to be filled.
    - The map contains information about businesses. One future application could be to 
    add another tag to such places in order to describe the type of business.
    - In order to collect momre information, games and tresure hunt like activities can be done. This way 
    society can combine and get together to enhance the quality and quantity of data.

Problems attached:
    - The overhead for creating a platform is a lot. It may require the overhaul of the existing methodology.
    - The cost associated with organising events could be a hindrance.
    - Manual labor is required to compltete the existing missing data for entries.
    
Benefits:
    - With more data, new patterns and insights can be generated.
   